# Deep Learning Bootcamp November 2017, GPU Computing for Data Scientists

<img src="../images/bcamp.png" align="center">

## 09 PyTorch Kaggle Image Data-set loading with CNN

Web: https://www.meetup.com/Tel-Aviv-Deep-Learning-Bootcamp/events/241762893/

Notebooks: <a href="https://github.com/QuantScientist/Data-Science-PyCUDA-GPU"> On GitHub</a>

*Shlomo Kashani*

<img src="../images/pt.jpg" width="35%" align="center">


# PyTorch Imports


In [6]:
# !pip install pycuda
%reset -f
import numpy
import numpy as np
from __future__ import print_function
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
import time
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (6, 6)      # setting default size of plots
import tensorflow as tf 
print("tensorflow:" + tf.__version__)
!set "KERAS_BACKEND=tensorflow"
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
# print('Active CUDA Device: GPU', torch.cuda.current_device())

# print ('Available devices ', torch.cuda.device_count())
# print ('Current cuda device ', torch.cuda.current_device())

# !pip install http://download.pytorch.org/whl/cu75/torch-0.2.0.post1-cp27-cp27mu-manylinux1_x86_64.whl
# !pip install torchvision 
# ! pip install cv2
# import cv2
# http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# https://www.bountysource.com/issues/44576966-a-tutorial-on-writing-custom-datasets-samplers-and-using-transforms
# https://medium.com/towards-data-science/my-first-kaggle-competition-9d56d4773607
# https://github.com/sohyongsheng/kaggle-planet-forest

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)


tensorflow:1.2.1
__Python VERSION: 2.7.6 (default, Oct 26 2016, 20:30:19) 
[GCC 4.8.4]
__pyTorch VERSION: 0.2.0_1
__CUDA VERSION
__CUDNN VERSION: 6021
__Number CUDA Devices: 0
__Devices


# No GPU ... ? 

```
---------------------------------------------------------------------------
OSError                                   Traceback (most recent call last)
<ipython-input-3-64c0769366fe> in <module>()
     36 print('__Number CUDA Devices:', torch.cuda.device_count())
     37 print('__Devices')
---> 38 call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
     39 print('Active CUDA Device: GPU', torch.cuda.current_device())
     40 
```

In [8]:
# Torch CPU
!pip install http://download.pytorch.org/whl/cu75/torch-0.2.0.post1-cp27-cp27mu-manylinux1_x86_64.whl
!pip install torchvision 


## CUDA Trick

In [9]:
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

# Setting up global variables

- Root folder
- Image folder
- Image Label folder

In [10]:
DATA_ROOT ='/root/data/amz/'
IMG_PATH = DATA_ROOT + '/train-jpg/'
IMG_EXT = '.jpg'
IMG_DATA_LABELS = DATA_ROOT + '/train_v2.csv'

# The Torch Dataset Class

In [21]:
try:
    from PIL import Image
except ImportError:
    import Image
    
class GenericImageDataset(Dataset):    

    def __init__(self, csv_path, img_path, img_ext, transform=None):
        
        t = time.time()        
        lgr.info('CSV path {}'.format(csv_path))
        lgr.info('IMG path {}'.format(img_path))        
        
        assert img_ext in ['.jpg']
        
        tmp_df = pd.read_csv(csv_path, header=None)
                        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df[0]        
#         self.X_train = self.X_train.ix[1:]
        
        self.y_train = self.mlb.fit_transform(tmp_df[1].str.split()).astype(np.float32)         
                
#         lgr.info("DF:\n" + str (self.X_train))
#         lgr.info ("self.y_train:\n" + str(self.y_train))

        lgr.info('[*]Dataset loading time {}'.format(time.time() - t))
        lgr.info('[*] Data size is {}'.format(len(self)))

    def __getitem__(self, index):
#         lgr.info ("__getitem__:" + str(index))
        path=self.img_path + self.X_train[index] + self.img_ext
#         lgr.info (" --- get item path:" + path)
        img = Image.open(path)
        img = img.convert('RGB')
        if self.transform is not None: # TypeError: batch must contain tensors, numbers, or lists; 
                                     #found <class 'PIL.Image.Image'>
            img = self.transform(img)
#             print (str (type(img))) # <class 'torch.FloatTensor'>                
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        l=len(self.X_train.index)
#         lgr.info ("Lenght:" +str(l))
        return (l)       

    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

    @staticmethod    
    def flaotTensorToImage(img, mean=0, std=1):
        """convert a tensor to an image"""
        img = np.transpose(img.numpy(), (1, 2, 0))
        img = (img*std+ mean)*255
        img = img.astype(np.uint8)    
        return img    
    
    @staticmethod
    def toTensor(img):
        """convert a numpy array of shape HWC to CHW tensor"""
        img = img.transpose((2, 0, 1)).astype(np.float32)
        tensor = torch.from_numpy(img).float()
        return tensor/255.0


# The Torch transforms.ToTensor() methood

- Converts: a PIL.Image or numpy.ndarray (H x W x C) in the range [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].

In [22]:
# transformations = transforms.Compose([transforms.ToTensor()])
transformations = transforms.Compose([transforms.Scale(32),transforms.ToTensor()])

# The Torch DataLoader Class

- Will load our GenericImageDataset
- Can be regarded as a list (or iterator, technically). 
- Each time it is invoked will provide a minibatch of (img, label) pairs.

In [23]:
dset_train = GenericImageDataset(IMG_DATA_LABELS,
                                 IMG_PATH,
                                 IMG_EXT,transformations)

train_loader = DataLoader(dset_train,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

INFO:__main__:CSV path /root/data/amz//train_v2.csv
INFO:__main__:IMG path /root/data/amz//train-jpg/
INFO:__main__:[*]Dataset loading time 0.292990922928
INFO:__main__:[*] Data size is 40479


# The NN model

- We will use a simple CNN with conv(3x3) -> bn -> relu -> pool(4x4) -> fc.

- In PyTorch, a model is defined by a subclass of nn.Module. It has two methods:

- `__init__: constructor. Create layers here. Note that we don't define the connections between layers in this function.`


- `forward(x): forward function. Receives an input variable x. Returns a output variable. Note that we actually connect the layers here dynamically.` 

In [24]:
# https://www.kaggle.com/mratsim/starting-kit-for-pytorch-deep-learning/code/notebook
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

model = Net() # On CPU
# model = Net().cuda() # On GPU

#  Loss and Optimizer

- Select a loss function and the optimization algorithm.

In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Start training in Batches

In [26]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            lgr.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))                   

In [27]:
for epoch in range(1, 2):
    train(epoch)

INFO:__main__:Train Epoch: 1 [0/40479 (0%)]	Loss: 0.697889
INFO:__main__:Train Epoch: 1 [6400/40479 (16%)]	Loss: 0.601731
INFO:__main__:Train Epoch: 1 [12800/40479 (32%)]	Loss: 0.281593
Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python2.7/dist-packages/torch/utils/data/dataloader.py", line 40, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-21-61579de41300>", line 38, in __getitem__
    img = Image.open(path)
  File "/usr/local/lib/python2.7/dist-packages/PIL/Image.py", line 2477, in open
    fp = builtins.open(filename, "rb")
KeyboardInterrupt


KeyboardInterrupt: 

In [29]:
%%bash
jupyter nbconvert \
    --to=slides \
    --reveal-prefix=https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.2.0/ \
    --output=py09.html \
    './09 PyTorch Kaggle Image Data-set loading with CNN'

[NbConvertApp] Converting notebook ./09 PyTorch Kaggle Image Data-set loading with CNN.ipynb to slides
[NbConvertApp] Writing 292502 bytes to ./py09.html.slides.html
